# Problem 1

## 1a

Consider the input string `"peanutrition"`, and suppose the cost function is in terms of unigrams. This means that the cost of a segementation of the string is just the sum of the costs associated with the individual words.

Assume the costs of words are as follows:
- `pea` costs 10
- `peanut` costs 8
- `nutrition costs 8
- all other words cost 99999

The greedy algorithm chooses to pick out `peanut` as the first word, since `pea` costs 10.
This leaves it having to segment the rest: `rition`. No matter what it does now, it will have at least 99999 cost added to its final solution. And `pea nutrition` is a segmentation that would have cost 18 but could not have been found by the greedy algorithm.

## 1b

How can we view of word segmentation as a _search_ problem?

Each state is a particular partial segmenetation. This can be represented by a list of indices of the last letters of words. If the input string is $s$ and the list of indices is $[i_0,\ldots,i_{n-1}]$, then the words are 

```s[:i[0]+1], s[i[0]+1:i[1]+1], ..., s[i[n-2]+1:i[n-1]+1]```

The indices have to be in increasing order, and they are constrained to the range `[0,len(s))`.

End states are ones for which the last index in the list is `len(s)-1`.

The initial state is `[]`, no words selected yet.

The possible actions from a state $[i_0,\ldots,i_{n-1}]$ are to append an index from the range `(i[n-1],len(s))`.

The cost of an appending an index `j` is the unigram cost of the new word that has been created.
This word is `s[i[-1]+1:j+1]`.

**WAIT** this is actually a terrible definition of "state"... we don't need an entire history to determine the cost of the next move. Remember: _A state is a summary of all the past actions which is sufficient to choose future actions optimally._

Each state will instead be a single "head" index, point to the most recently selected word head.

The initial state is 0. End state could be a special end token, or it could just be the one-past-the-end index `len(s)`, where `s` is the input string.

The states accessible from state `i` are all of the indices in the range `[i+1,len(s)]`.
For each for $j\in [i+1,\text{len}(s)]$, there is exactly one action $i\to j$ and its cost is the unigram cost of the word `s[i:j]`.

In [1]:
from util import *

In [2]:
# Testing out the toy example we are given

nl = NumberLineSearchProblem()
ucs = UniformCostSearch(1) # adjust verbosity here
ucs.solve(nl)

numStatesExplored = 31
totalCost = 20
actions = ['East', 'East', 'East', 'East', 'East', 'East', 'East', 'East', 'East', 'East']


In [3]:
gs = GridSearchProblem(4,2,3)
ucs = UniformCostSearch(1)
ucs.solve(gs)

numStatesExplored = 16
totalCost = 10
actions = ['North', 'North', 'West', 'West', 'West']


In [11]:
from submission import *

peanut_unigram_cost_dict = {
    "pea":10,
    "peanut":8,
    "nutrition":8,
}
peanut_unigram_cost = lambda s : peanut_unigram_cost_dict.get(s,99999)

ucs = UniformCostSearch(3)
ucs.solve(SegmentationProblem("peanutrition",peanut_unigram_cost))
print(' '.join(ucs.actions))

Exploring 0 with pastCost 0
  Action p => 1 with cost 0 + 99999
  Action pe => 2 with cost 0 + 99999
  Action pea => 3 with cost 0 + 10
  Action pean => 4 with cost 0 + 99999
  Action peanu => 5 with cost 0 + 99999
  Action peanut => 6 with cost 0 + 8
  Action peanutr => 7 with cost 0 + 99999
  Action peanutri => 8 with cost 0 + 99999
  Action peanutrit => 9 with cost 0 + 99999
  Action peanutriti => 10 with cost 0 + 99999
  Action peanutritio => 11 with cost 0 + 99999
  Action peanutrition => 12 with cost 0 + 99999
Exploring 6 with pastCost 8
  Action r => 7 with cost 8 + 99999
  Action ri => 8 with cost 8 + 99999
  Action rit => 9 with cost 8 + 99999
  Action riti => 10 with cost 8 + 99999
  Action ritio => 11 with cost 8 + 99999
  Action rition => 12 with cost 8 + 99999
Exploring 3 with pastCost 10
  Action n => 4 with cost 10 + 99999
  Action nu => 5 with cost 10 + 99999
  Action nut => 6 with cost 10 + 99999
  Action nutr => 7 with cost 10 + 99999
  Action nutri => 8 with cost 10 

In [13]:
from submission import *

peanut_unigram_cost_dict = {
    "pea":10,
    "peanut":8,
    "nutrition":8,
}
peanut_unigram_cost = lambda s : peanut_unigram_cost_dict.get(s,5*len(s))

ucs = UniformCostSearch(1)
ucs.solve(SegmentationProblem("peanutritionnutritionalpeanuts",peanut_unigram_cost))
print(' '.join(ucs.actions))

numStatesExplored = 27
totalCost = 49
actions = ['pea', 'nutrition', 'nutrition', 'al', 'peanut', 's']
pea nutrition nutrition al peanut s


# Problem 2
## 2a

Consider the input string `t ck dgh`, perhaps preceded by some other stuff.
Suppose that `possibleFills` gives these limited options:
- `possibleFills['t'] = ['eat']`
- `possibleFills['ck'] = ['cookie', 'cake']`
- `possibleFills['dgh'] = ['dough']`

and suppose that the bigram cost function is such that `eat cookie` is cheaper than `eat cake`, but `cookie dough` is _much much_ cheaper than `cake dough`.

Then the greedy algorithm will produce `eat cake dough` while the optimal solution is `eat cookie dough`.

## 2b

We have `queryWords`, a list of vowel-less words, and we have the functions `bigramCost` and `possibleFills`.

How do we formulate this as a search problem? i.e. what will be the state?

A state will be a pair consisting of a word and an index. The word is the "previously selected" word in the vowel insertion process, and the index points to a word in `queryWords`.

The initial state is `(wordsegUtil.SENTENCE_BEGIN,0)`

Given state `(word,i)` the possible actions are different choices of elements of `possibleFills(queryWords[i])`. If one chooses the element `newWord`, then the new state is `(newWord,i+1)` and the costof that is `bigramCost(word,newWord)`.

An end state is any state of the form `( _ , len(queryWords) )`.


# Problem 3

Now we are given `query`, `bigramCost`, and `possibleFills`.

We need to both insert spaces and insert vowels into the query string.

What will be the state?

A state will be a pair consisting of a word and an index. The word is the "previously constructed" word and the index points to a character in `query`.

The initial state is `(wordsegUtil.SENTENCE_BEGIN, 0)` and an end state is any state of the form `( _ , len(query) )`.

Given a state `(word,i)`, the reachable states are

``` {  (filledNextWord, j) |  j in [i+1,len(query)]  and  filledNextWord in possibleFills(query[i:j])  }```

and the cost of going from `(word,i)` to `(filledNextWord, j)` is `bigramCost(word,filledNextWord)`.

## 3c

Given $b(w',w)$, let's come up with a useful $f_b(w)$.

... bleh... let's actually pretend we have our $f_b(w)$ and see what's next:

Define a relaxed version of the problem of parts (3a) and (3b) as follows:
- Simplify the state `(word,i)` to just `i`. States are just indices in the `[0,len(query)]`.
- The start state is `0` and the end state is `len(query)`.
- Given a state `i` the potentially reachable states are all the `j` in `[i+1,len(query)]`, however there can be many, or no, actions $i\to j$. There is one action $i\to j$ for each element $w$ of `possibleFills(query[i:j])`, and the cost of that action is $f_b(w)$.

Is this new problem (let's call it $P_\text{rel}$) really a _relaxation_ of the original problem (let's call it $P$)? All states `(word,i)` from $P$ get mapped to just `i`, and the edge $(w',i)\to (w,j)$ with cost $b(w',w)$ becomes an edge $i\to j$ with cost $f_b(w)$. In order for the image problem $P_\text{rel}$ to be a relaxation of $P$, we must have

$$ \text{Cost}_{P_\text{rel}}(i\xrightarrow{w} j) \leq \text{Cost}_P((w',i)\xrightarrow{w}(w,j)) $$

That is, we must have

$$ f_b(w) \leq b(w',w). $$

(If we want this to work independently of input `query` then the inequality had better just hold for all words $w,w'$.)

_Now_ I have some idea of how to pick $f_b(w)$:
$$ f_b(w) = \min_{w'} b(w',w). $$

## 3d

UCS is a special case of A*, where the heuristic function is taken to be $0$.

BFS is a special case of UCS where the graph is acyclic and the costs are all the same.
To see this, note that the frontier of UCS would become the "depth level" being explored in BFS, and you would pop everything at one level before moving on to the next.